# Controlling Model Output

## Setup

In [1]:
# Load env variables
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Create API Client
from anthropic import Anthropic

client = Anthropic()
model = "claude-sonnet-4-0"

In [11]:
# Helper functions
def add_user_message(messages, text):
    user_message = {
        "role": "user",
        "content": text
    }
    messages.append(user_message)
    

def add_assistant_message(messages, text):
    assistant_message = {
        "role": "assistant",
        "content": text
    }
    messages.append(assistant_message)
    

def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences
    }

    if system:
        params["system"] = system
    
    message = client.messages.create(**params)
    return message.content[0].text

## Message Prefilling

In [6]:
# Default behaviour
messages = []

add_user_message(
    messages, 
    "Is tea or coffee better at breakfast?"
)
answer = chat(messages)

answer

"Both tea and coffee can be great breakfast choices, and it really depends on your personal preferences and needs:\n\n**Coffee might be better if you:**\n- Need a stronger caffeine boost to wake up\n- Enjoy bold, rich flavors\n- Want something that pairs well with hearty breakfast foods\n- Prefer the ritual of brewing/preparing it\n\n**Tea might be better if you:**\n- Prefer a gentler, more gradual energy boost\n- Want something lighter on your stomach\n- Enjoy the variety of flavors and types available\n- Are looking for additional health benefits (antioxidants, etc.)\n- Want to stay more hydrated\n\n**Consider also:**\n- Your caffeine tolerance and sensitivity\n- Any stomach sensitivities (tea is generally gentler)\n- Your morning schedule (coffee is often quicker to prepare)\n- Cultural or personal preferences\n\nMany people find that what works best can even vary day to day based on their mood, what they're eating, or how much sleep they got. Have you noticed a preference in your o

In [7]:
# Message Prefilling
messages = []

add_user_message(
    messages, 
    "Is tea or coffee better at breakfast?"
)
add_assistant_message(
    messages,
    "Coffee is better because"
)
answer = chat(messages)

answer

" it has more caffeine, which helps you feel alert in the morning. Tea is usually better throughout the day because it has less caffeine and fewer side effects. Both are healthy choices though - it really comes down to your personal preference and caffeine tolerance.\n\nSome people prefer tea at breakfast because:\n- It's gentler on an empty stomach\n- Green tea provides antioxidants\n- It offers a calmer energy boost\n\nOthers prefer coffee because:\n- Higher caffeine content for alertness\n- Familiar morning routine\n- Pairs well with breakfast foods\n\nWhat matters most is what makes you feel good and fits your lifestyle!"

## Stop Sequences

In [9]:
# Default behaviour
messages = []

add_user_message(messages, "Count from 1 to 10")
answer = chat(messages)

answer

'1, 2, 3, 4, 5, 6, 7, 8, 9, 10'

In [12]:
messages = []

add_user_message(messages, "Count from 1 to 10")
answer = chat(messages, stop_sequences=[", 5"])

answer

'1, 2, 3, 4'

## Structured Data

In [17]:
messages = []

add_user_message(messages, "Generate a very short event bridge rule as json")
add_assistant_message(messages, "```json")

text = chat(messages, stop_sequences=["```"])
text

'\n{\n  "Name": "OrderProcessingRule",\n  "EventPattern": {\n    "source": ["myapp.orders"],\n    "detail-type": ["Order Placed"]\n  },\n  "Targets": [\n    {\n      "Id": "1",\n      "Arn": "arn:aws:lambda:us-east-1:123456789012:function:ProcessOrder"\n    }\n  ],\n  "State": "ENABLED"\n}\n'

In [18]:
import json

json.loads(text.strip())

{'Name': 'OrderProcessingRule',
 'EventPattern': {'source': ['myapp.orders'], 'detail-type': ['Order Placed']},
 'Targets': [{'Id': '1',
   'Arn': 'arn:aws:lambda:us-east-1:123456789012:function:ProcessOrder'}],
 'State': 'ENABLED'}

## Exercise

In [19]:
messages = []

prompt = """ 
Generate three different sample AWS CLI commands. Each should be very short.
"""

add_user_message(messages, prompt)

text = chat(messages)
text.strip()

'Here are three short AWS CLI commands:\n\n1. **List S3 buckets:**\n   ```bash\n   aws s3 ls\n   ```\n\n2. **Describe EC2 instances:**\n   ```bash\n   aws ec2 describe-instances\n   ```\n\n3. **Get caller identity:**\n   ```bash\n   aws sts get-caller-identity\n   ```'

In [20]:
from IPython.display import Markdown

Markdown(text)

Here are three short AWS CLI commands:

1. **List S3 buckets:**
   ```bash
   aws s3 ls
   ```

2. **Describe EC2 instances:**
   ```bash
   aws ec2 describe-instances
   ```

3. **Get caller identity:**
   ```bash
   aws sts get-caller-identity
   ```

In [24]:
messages = []

prompt = """ 
Generate three different sample AWS CLI commands. Each should be very short.
"""

add_user_message(messages, prompt)
add_assistant_message(messages, "Here are all three commands in a single block without ant comments:\n```bash")

text = chat(messages, stop_sequences=["```"])
text.strip()

'aws s3 ls\naws ec2 describe-instances\naws lambda list-functions'